In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
import numpy as np
import os


In [2]:
# Loading datasets; identical to notebook 01
DATA_DIR = '../data/asl_alphabet_train/asl_alphabet_train'
TEST_DATA_DIR = '../data/asl_alphabet_test/asl_alphabet_test'

IMAGE_SIZE = (96, 96)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
SEED = 123
NUM_CLASSES = 29  # A-Z, del, nothing, space

print("Loading training and validation datasets...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

class_names = train_ds.class_names
print(f"Found {len(class_names)} classes: {class_names}")


Loading training and validation datasets...
Found 86912 files belonging to 29 classes.
Using 69530 files for training.
Found 86912 files belonging to 29 classes.
Using 17382 files for validation.
Found 29 classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [3]:
# Data augmentation and normalization
print("Defining augmentation and normalization layers...")

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomContrast(0.1),
    ],
    name="data_augmentation"
)

normalization_layer = layers.Rescaling(1./255)

# Apply augmentation to training data, normalization to both
print("Applying processing and optimizing datasets...")

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("Data processing pipeline complete and optimized.")

Defining augmentation and normalization layers...
Applying processing and optimizing datasets...
Data processing pipeline complete and optimized.


In [4]:
#Loading in ResNet50 and then freezing (not changing the weights) of the base model
base_model = ResNet50(
    weights='imagenet',     
    include_top=False,        
    input_shape=(96, 96, 3)  
)

base_model.trainable = False
print(f"  Number of layers: {len(base_model.layers)}")
print(f"  Total parameters: {base_model.count_params():,}")
print(f"  Trainable layers: {sum([1 for layer in base_model.layers if layer.trainable])}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
  Number of layers: 175
  Total parameters: 23,587,712
  Trainable layers: 0


In [5]:
#Running images through the base model
inputs = keras.Input(shape=(96, 96, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs, name='asl_resnet50')
model.summary()


Model: "asl_resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 3, 3, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,853,725 (90.99 MB)

 Trainable params: 266,013 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
#Preparing the model for training
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),  
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Phase 1: Training New Layers (Base Model Frozen)


In [7]:
callbacks_list = [
    #Early stopping to save time
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    #Slowing down learning rate when stuck
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    # Saving best model
    callbacks.ModelCheckpoint(
        'best_asl_resnet50_phase1.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]



In [ ]:
#Training the new layers
history_phase1 = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=callbacks_list,
    verbose=1
)

Epoch 1/10
1183/2173 ━━━━━━━━━━━━━━━━━━━━ 1:23 85ms/step - accuracy: 0.0493 - loss: 3.3643

## Phase 2: Fine-Tuning Entire Model


In [ ]:
#Unfreezing the base model
base_model.trainable = True
for layer in base_model.layers[:143]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),  # 10x lower LR
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Total trainable parameters now: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")


In [ ]:
#Updating callbacks for Phase 2
callbacks_list_phase2 = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        'best_asl_resnet50_phase2.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Continue training with fine-tuning
history_phase2 = model.fit(
    train_ds,
    epochs=10,  
    validation_data=val_ds,
    callbacks=callbacks_list_phase2,
    verbose=1,
    initial_epoch=len(history_phase1.epoch) 
)

print("\nFine-tuning complete!")

## Training Results


In [ ]:
# Combine training histories
def combine_histories(h1, h2):
    """Combine two training histories"""
    combined = {}
    for key in h1.keys():
        combined[key] = h1[key] + h2[key]
    return combined

history = combine_histories(history_phase1.history, history_phase2.history)

# Print final metrics 
# Phase 1 results
print("\nPhase 1 (Frozen Base Model):")
print(f"  Training Accuracy: {history_phase1.history['accuracy'][-1]:.4f} ({history_phase1.history['accuracy'][-1]*100:.2f}%)")
print(f"  Validation Accuracy: {history_phase1.history['val_accuracy'][-1]:.4f} ({history_phase1.history['val_accuracy'][-1]*100:.2f}%)")
print(f"  Training Loss: {history_phase1.history['loss'][-1]:.4f}")
print(f"  Validation Loss: {history_phase1.history['val_loss'][-1]:.4f}")

# Phase 2 results
print("\nPhase 2 (Fine-Tuned Model):")
print(f"  Training Accuracy: {history_phase2.history['accuracy'][-1]:.4f} ({history_phase2.history['accuracy'][-1]*100:.2f}%)")
print(f"  Validation Accuracy: {history_phase2.history['val_accuracy'][-1]:.4f} ({history_phase2.history['val_accuracy'][-1]*100:.2f}%)")
print(f"  Training Loss: {history_phase2.history['loss'][-1]:.4f}")
print(f"  Validation Loss: {history_phase2.history['val_loss'][-1]:.4f}")


In [ ]:
# Save the final model
model.save('asl_resnet50_final.h5')
print("Model saved as 'asl_resnet50_final.h5'")

# Also save the best model if it exists
if os.path.exists('best_asl_resnet50_phase2.h5'):
    print("Best model (from checkpoint) saved as 'best_asl_resnet50_phase2.h5'")
